# Descargar Librerias

In [ ]:
#Descargar siempre cuando usen Colab
#!pip install praw tqdm

# Iniciar el Extractor y Completar debajo

In [1]:
import praw
import pandas as pd
import os
import time
from tqdm import tqdm

# Configura tu instancia de PRAW con tus credenciales de la aplicación y un user_agent único
reddit_config = {
    'client_id': '9zK4SSfBSCMJPnpX1rbg4Q',
    'client_secret': 'CI5_-mXS5NGVZMvNjmOb6kyE60fG_A',
    'user_agent': 'MiScriptRedditBot/v1.7 (by Sheincito en Reddit)'
}

# Define el subreddit a extraer
subreddit_to_extract = input("Ingrese el nombre del subreddit: ")

# Define el filtro para las publicaciones
filtro = input("Ingrese el filtro para las publicaciones (deje en blanco para no filtrar): ")

# Nombre del archivo CSV basado en el subreddit extraído
csv_filename = f"extractor_{subreddit_to_extract}.csv"

# Función para cargar los títulos existentes del archivo CSV
def cargar_titulos_existentes(filename, subreddit_name):
    if os.path.isfile(filename):  
        try:
            df = pd.read_csv(filename)
            if df.empty:
                return df
            else:
                return set(df['Reddit'])
        except pd.errors.EmptyDataError:
            return df
    else: 
        return pd.DataFrame(columns=[subreddit_name])

# Función para recopilar publicaciones
def recopilar_publicaciones(subreddit_name, filename, reddit, filtro=None):
    # Cargar los títulos existentes del archivo CSV
    titulos_existentes = cargar_titulos_existentes(filename, subreddit_name)
    
    # Obtiene el subreddit
    subreddit = reddit.subreddit(subreddit_name)
    
    # Lista para almacenar las publicaciones
    posts = []
    
    # Convertir el filtro a minúsculas si está presente
    filtro = filtro.lower() if filtro else None
    
    # Dividir el filtro en palabras si contiene varias palabras
    filtro_palabras = filtro.split() if filtro else None
    
    # Bucle para recopilar publicaciones
    for submission in tqdm(subreddit.new(limit=None), desc='Recopilando publicaciones'):
        # Convertir el título a minúsculas para comparación sin distinción de mayúsculas y minúsculas
        titulo_lower = submission.title.lower()
        
        # Aplicar filtro si está presente
        if filtro:
            # Verificar si alguna de las palabras del filtro está en el título de la publicación
            if not any(word in titulo_lower for word in filtro_palabras):
                continue
        
        # Verifica si el título de la publicación ya está en los títulos existentes
        if submission.title not in titulos_existentes:
            # Agrega el título a la lista de títulos y al conjunto de títulos existentes
            titulos_existentes.add(submission.title)
            
            # Agrega el título y el cuerpo de la publicación a la lista de publicaciones
            posts.append({'Reddit': submission.title, 'Body': submission.selftext})
    
            # Convierte la lista de publicaciones en un DataFrame
            df = pd.DataFrame(posts)
    
            # Guarda las publicaciones en un archivo CSV (modo 'a' para agregar datos sin sobrescribir)
            df.to_csv(filename, index=False, mode='a', header=not os.path.exists(filename))
            
            # Espera 1 segundo antes de procesar la siguiente publicación
            time.sleep(1)

# Configura PRAW con la configuración especificada
reddit = praw.Reddit(**reddit_config)

# Llama a la función para recopilar publicaciones y guardarlas en un archivo CSV
recopilar_publicaciones(subreddit_to_extract, csv_filename, reddit, filtro)

# Cuenta las líneas del archivo CSV para determinar cuántas publicaciones se extrajeron
with open(csv_filename, 'r', encoding='utf-8') as file:
    total_publicaciones = sum(1 for line in file)

# Mensaje de finalización
print(f"Se han extraído y guardado exitosamente {total_publicaciones} publicaciones del subreddit '{subreddit_to_extract}'.")
print("Proceso finalizado.")



Recopilando publicaciones: 977it [02:07,  7.65it/s]

Se han extraído y guardado exitosamente 8661 publicaciones del subreddit 'argentina'.
Proceso finalizado.


# Cargamos el DF generado para probar

In [2]:
# Carga el archivo CSV que contiene los datos del DataFrame
df = pd.read_csv(csv_filename)
pd.options.display.max_colwidth = 10000

## Aqui borre los nulos por que cuando hay imagenes no las guarda y queda como NaN en el Body

In [10]:
df2=df.dropna(subset=['Body'])

In [11]:
len(df2)

448

In [13]:
len(df2['Reddit'].unique())

8